In [ ]:
import requests

response = requests.post('https://fbrapi.com/generate_api_key')
api_key = response.json()['api_key']

In [ ]:
# This function creates a dataframe with 3 columns (Team, Penalty Kick Attempts,
# Penalty Kick Scored) with Team as the index.

import time
import pandas as pd

def create_df(league: int, year: str):
  # we will be running this multiple times - FBReference has a limit of 6 seconds between requests.
  t = 6.1
  time.sleep(t)

  url = "https://fbrapi.com/team-season-stats"
  params = {
    "league_id": league,
    "season_id": year,
  }
  response = requests.get(url=url, params=params, headers={"X-API-Key": api_key})
  response.raise_for_status()
  data = response.json()

  for meta in data:
    data_simplified = data[meta]

  pk_made = []
  pk_attempts = []
  team_names = []

  for goal in pd.DataFrame(data_simplified)['stats']:
    pk_made.append(goal['stats']['ttl_pk_made'])

  for attempt in pd.DataFrame(data_simplified)['stats']:
    pk_attempts.append(attempt['stats']['ttl_pk_att'])

  for team in pd.DataFrame(data_simplified)['meta_data']:
    team_names.append(team['team_name'])

  dict = {'team': team_names, 'pk attempts': pk_attempts, 'pk made': pk_made}

  return pd.DataFrame(dict).set_index('team')


df_pl2122 = create_df(9, '2021-2022')   #the English Premier League code is 9

print(f'\n Example DataFrame \nPremier League 2021-2022 Season:\n {df_pl2122}')



 Example DataFrame 
Premier League 2021-2022 Season:
                  pk attempts  pk made
team                                 
Arsenal                    8        5
Aston Villa                3        3
Brentford                  6        6
Brighton                   7        4
Burnley                    2        1
Chelsea                    9        8
Crystal Palace             8        6
Everton                    7        6
Leeds United               5        5
Leicester City             2        2
Liverpool                  8        7
Manchester City            9        7
Manchester Utd             5        3
Newcastle Utd              3        3
Norwich City               3        3
Southampton                4        4
Tottenham                  5        5
Watford                    2        1
West Ham                   6        4
Wolves                     1        1


In [ ]:
# To get multiple dataframes lets run the function 'create_df' a few more times.
# This will take a minute approximately 5 minutes for all the requests to go through.

league_codes = {
    "English Premier League": 9,
    "German Bundesliga": 20,
    "French Ligue 1": 13,
    "Spanish La Liga": 12,
    "Italian Serie A": 11,
}

seasons = ["2020-2021", "2021-2022", "2022-2023", "2023-2024", "2024-2025"]

def merge(input_df_list):

  merged_df = pd.concat(input_df_list, join='outer').groupby('team').sum()
  merged_df['conversion rate'] = (merged_df['pk made']/merged_df['pk attempts']).round(2)
  merged_df.sort_values(by='conversion rate', ascending=False)
  return merged_df

# English Premier League
pl_20 = create_df(9, seasons[0])
pl_21 = create_df(9, seasons[1])
pl_22 = create_df(9, seasons[2])
pl_23 = create_df(9, seasons[3])
pl_24 = create_df(9, seasons[4])

df_pl = [pl_20, pl_21, pl_22, pl_23, pl_24]

# German Bundesliga
bund_20 = create_df(20, seasons[0])
bund_21 = create_df(20, seasons[1])
bund_22 = create_df(20, seasons[2])
bund_23 = create_df(20, seasons[3])
bund_24 = create_df(20, seasons[4])

df_bund = [bund_20, bund_21, bund_22, bund_23, bund_24]

# Serie A
sa_20 = create_df(11, seasons[0])
sa_21 = create_df(11, seasons[1])
sa_22 = create_df(11, seasons[2])
sa_23 = create_df(11, seasons[3])
sa_24 = create_df(11, seasons[4])

df_sa = [sa_20, sa_21, sa_22, sa_23, sa_24]

# French Ligue 1
ligue_20 = create_df(13, seasons[0])
ligue_21 = create_df(13, seasons[1])
ligue_22 = create_df(13, seasons[2])
ligue_23 = create_df(13, seasons[3])
ligue_24 = create_df(13, seasons[4])

df_ligue = [ligue_20, ligue_21, ligue_22, ligue_23, ligue_24]

# Spanish La Liga
liga_20 = create_df(12, seasons[0])
liga_21 = create_df(12, seasons[1])
liga_22 = create_df(12, seasons[2])
liga_23 = create_df(12, seasons[3])
liga_24 = create_df(12, seasons[4])

df_liga = [liga_20, liga_21, liga_22, liga_23, liga_24]

print('Example of a merged and sorted table - Spanish Top League (2020-2025)')
merged_liga = merge(df_liga)


Example of a merged and sorted table - Spanish Top League (2020-2025)
                 pk attempts  pk made  conversion rate
team                                                  
Alavés                    33       23            0.697
Almería                    5        4            0.800
Athletic Club             30       18            0.600
Atlético Madrid           26       21            0.808
Barcelona                 32       23            0.719
Betis                     39       31            0.795
Celta Vigo                30       23            0.767
Cádiz                     18       14            0.778
Eibar                      9        4            0.444
Elche                      9        7            0.778
Espanyol                  15       15            1.000
Getafe                    30       22            0.733
Girona                    22       19            0.864
Granada                   18       11            0.611
Huesca                     4        3            0

In [ ]:
merged_pl = merge(df_pl)
merged_bund = merge(df_bund)
merged_ligue = merge(df_ligue)
merged_liga = merge(df_liga)
merged_sa = merge(df_sa)

merged_pl['league'] = 'English Premier League'
merged_bund['league'] = 'German Bundesliga'
merged_ligue['league'] = 'French Ligue 1'
merged_liga['league'] = 'Spanish La Liga'
merged_sa['league'] = 'Italian Serie A'

all_merged = [merged_pl, merged_bund, merged_ligue, merged_liga, merged_sa]
all_leagues = pd.concat(all_merged, join='outer')
print(all_leagues.sort_values(by='conversion rate', ascending=False))

              pk attempts  pk made  conversion rate                  league
team                                                                       
Wolves                 12       12            1.000  English Premier League
Norwich City            3        3            1.000  English Premier League
Luton Town              5        5            1.000  English Premier League
Ipswich Town            2        2            1.000  English Premier League
Crotone                 9        9            1.000         Italian Serie A
...                   ...      ...              ...                     ...
Watford                 2        1            0.500  English Premier League
St. Pauli               2        1            0.500       German Bundesliga
Schalke 04              8        4            0.500       German Bundesliga
Eibar                   9        4            0.444         Spanish La Liga
Como                    2        0            0.000         Italian Serie A

[132 rows x

In [52]:
filtered_by_attempts = all_leagues.loc[all_leagues['pk attempts'] > 5]
print(filtered_by_attempts.sort_values(by='conversion rate', ascending=True).head(10))

               pk attempts  pk made  conversion rate                  league
team                                                                        
Eibar                    9        4            0.444         Spanish La Liga
Schalke 04               8        4            0.500       German Bundesliga
Levante                 19       11            0.579         Spanish La Liga
Athletic Club           30       18            0.600         Spanish La Liga
Granada                 18       11            0.611         Spanish La Liga
Fulham                  21       13            0.619  English Premier League
Stuttgart               30       19            0.633       German Bundesliga
Ajaccio                  9        6            0.667          French Ligue 1
Metz                    18       12            0.667          French Ligue 1
Sampdoria               12        8            0.667         Italian Serie A


In [ ]:
pk_goals = all_leagues['pk made'].sum()
total_attempts = all_leagues['pk attempts'].sum()



print(f'out of {total_attempts} attempts from a PK, {pk_goals} goals have been scored')
print('The PK conversion rate over the top 5 leagues in Europe in the last 5 years is...')
print(pk_goals / total_attempts)

out of 2910 attempts from a PK, 2312 goals have been scored
The PK conversion rate over the top 5 leagues in Europe in the last 5 years is...
0.7945017182130584


In [53]:
pd.pivot_table(filtered_by_attempts, values='conversion rate', index='league', aggfunc='median')

,conversion rate
league,
English Premier League,0.833
French Ligue 1,0.841
German Bundesliga,0.784
Italian Serie A,0.786
Spanish La Liga,0.767


In [49]:
pivot_leagues = pd.pivot_table(all_leagues, values=['pk attempts','pk made'], index='league', aggfunc='sum')
pivot_leagues['conversion rate'] = (pivot_leagues['pk made']/pivot_leagues['pk attempts']).round(2)
pivot_leagues.sort_values(by='conversion rate', ascending=False)

,pk attempts,pk made,conversion rate
league,,,
French Ligue 1,645,533,0.83
English Premier League,517,425,0.82
Italian Serie A,635,504,0.79
German Bundesliga,488,377,0.77
Spanish La Liga,625,473,0.76


In [ ]:
# Check outliers in the data